# HW5 — Problem 3: Adiabatic flame temperature of acetylene

Adiabatic energy balance:
0 = ΔH₍₂₉₈₎ + ∫ cp_products dT, with products as:
- O₂ only: 2 CO₂ + H₂O
- Air:     2 CO₂ + H₂O + 9.41 N₂ (ballast)


In [ ]:

import numpy as np

# Formation enthalpies at 298 K (J/mol)
H_C2H2 = +226700.0
H_CO2  = -393500.0
H_H2O  = -241800.0

dH298 = 2*H_CO2 + H_H2O - H_C2H2  # J/mol

# cp polynomials (J/mol/K): a + b T + c T^{-2}
def int_cp_poly(a,b,c, T1,T2):
    return a*(T2-T1) + 0.5*b*(T2**2 - T1**2) - c*(1.0/T2 - 1.0/T1)

def int_cp_CO2(T1,T2): return int_cp_poly(44.14, 9.04e-3, 8.54e5, T1,T2)
def int_cp_H2O(T1,T2): return int_cp_poly(30.00, 10.71e-3, -0.33e5, T1,T2)
def int_cp_N2 (T1,T2): return 27.87*(T2-T1) + 0.5*4.27e-3*(T2**2 - T1**2)

T1 = 298.0

def balance_O2(T):
    return dH298 + (2*int_cp_CO2(T1,T) + int_cp_H2O(T1,T))

def balance_Air(T):
    return dH298 + (2*int_cp_CO2(T1,T) + int_cp_H2O(T1,T) + 9.41*int_cp_N2(T1,T))

def solve_bisect(f, lo, hi, tol=1e-6, maxit=200):
    f_lo, f_hi = f(lo), f(hi)
    if f_lo*f_hi > 0:
        raise RuntimeError("Bisection bracket does not straddle a root")
    for _ in range(maxit):
        mid = 0.5*(lo+hi)
        f_mid = f(mid)
        if abs(f_mid) < 1e-3 or (hi-lo) < tol:
            return mid
        if f_lo*f_mid <= 0:
            hi, f_hi = mid, f_mid
        else:
            lo, f_lo = mid, f_mid
    return 0.5*(lo+hi)

T_ad_O2  = solve_bisect(balance_O2, 500.0, 8000.0)
T_ad_air = solve_bisect(balance_Air, 500.0, 8000.0)

print(f"T_ad (O2)  = {T_ad_O2:.0f} K")
print(f"T_ad (air) = {T_ad_air:.0f} K")
